<h1><center> ASDS 5303 Final Project Assignment #3 Dataset 1: Drug SMILES Strings and Classifications - Deep Network Analysis </center></h1>

## Group Members
### Henry Berrios #1001392315
### LeMaur Kydd #1001767382

In [ ]:
# importing libraries (same as previous assignments, partially filled by google colab autofill)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import pandas as pd
import numpy as np
from scipy.io import wavfile

import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from google.colab import drive

# **A. Create a Deep Neural Network**

In [ ]:
# using cuda if we can
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# loading pre processed data
X_train = torch.load('/content/drive/MyDrive/X_train_tensor_d1.pt').to(device)
X_test = torch.load('/content/drive/MyDrive/X_test_tensor_d1.pt').to(device)
y_train = torch.load('/content/drive/MyDrive/y_train_tensor_d1.pt').to(device)
y_test = torch.load('/content/drive/MyDrive/y_test_tensor_d1.pt').to(device)

<ipython-input-3-d0af1adbe808>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train = torch.load('/content/drive/MyDrive/X_train_tensor_d1.pt').to(device)
<ipython-input

In [ ]:
# checking shape of tensors
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

torch.Size([5548, 399, 128])
torch.Size([5548, 5])
torch.Size([1387, 399, 128])
torch.Size([1387, 5])


In [ ]:
# defining a neural network (debugging assisted by OpenAI's ChatGPT)
class DeepNetwork(nn.Module):
  def __init__(self, input_size, hidden_layers, hidden_units, output_size):
    super(DeepNetwork, self).__init__()
    layers = []
    layers.append(nn.Linear(input_size, hidden_units))
    layers.append(nn.ReLU())

    for _ in range(hidden_layers - 1):
      layers.append(nn.Linear(hidden_units, hidden_units))
      layers.append(nn.ReLU())

    layers.append(nn.Linear(hidden_units, output_size))
    self.model = nn.Sequential(*layers)

  def forward(self, x):
    x = x.view(x.size(0), -1) # flattens the output
    return self.model(x)

In [ ]:
# testing on small sample
sample_inputs = X_train[:100].to(device)

In [ ]:
# passing through the model
deep_test_model = DeepNetwork(input_size = sample_inputs.shape[1] * sample_inputs.shape[2], hidden_layers = 2, hidden_units = 128, output_size = y_train.shape[1]).to(device)

sample_output = deep_test_model(sample_inputs)

print(sample_output.shape)

torch.Size([100, 5])


In [ ]:
# checking for training errors (large portions of code are autofilled by google colab's gemini)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(deep_test_model.parameters(), lr=0.001)

sample_train = y_train[:100].to(device)
optimizer.zero_grad()
outputs = deep_test_model(sample_inputs)
loss = criterion(outputs, sample_train)
loss.backward()
optimizer.step()

print(loss.item())

1.662506103515625


In [ ]:
# training on small sample (large portions of code are autofilled by google colab's gemini)
small_train_loader = DataLoader(TensorDataset(X_train[:10], y_train[:10]), batch_size=2, shuffle=True)

for epoch in range(10):
  deep_test_model.train()
  epoch_loss = 0

  for batch_x, batch_y in small_train_loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    optimizer.zero_grad()

    outputs = deep_test_model(batch_x)
    loss = criterion(outputs, batch_y)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

  print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(small_train_loader)}")

Epoch 1, Loss: 6.954846954345703
Epoch 2, Loss: 2.1673967123031614
Epoch 3, Loss: 0.9131421744823456
Epoch 4, Loss: 0.40645577274262906
Epoch 5, Loss: 0.22919289506971835
Epoch 6, Loss: 0.2770965125411749
Epoch 7, Loss: 0.2771903851957177
Epoch 8, Loss: 0.013115176698192954
Epoch 9, Loss: 0.023283789050765336
Epoch 10, Loss: 0.011135860963258892


# **B. Optimize Network Depth and Width**

In [ ]:
# optimizing network depth and width (large portions of code are autofilled by google colab's gemini)
def train_deep_model(model, train_loader, test_loader, epochs = 20, lr = 0.001):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=lr)

  train_losses = []
  test_losses = []
  for epoch in range(epochs):
    model.train()
    train_loss = 0

    for batch_x, batch_y in train_loader:
      batch_x, batch_y = batch_x.to(device), batch_y.to(device)
      optimizer.zero_grad()
      outputs = model(batch_x)
      loss = criterion(outputs, batch_y)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()

    model.eval()
    test_loss = 0
    with torch.no_grad():
      for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        test_loss += loss.item()

    train_loss /= len(train_loader)
    test_loss /= len(test_loader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Test Loss: {test_loss}")

  return train_losses, test_losses

In [ ]:
# testing optimization
batch_size = 256 # trying different batch sizes

In [ ]:
# creating dataset to use for training and testing
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# creating dataloaders for batch processing
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
# experimenting with depths and widths
hidden_layers_list = [1, 2, 3]
hidden_units_list = [32, 64, 128]

In [ ]:
# initializing variables to track best model configuration
best_loss = float('inf')
best_config = None

In [ ]:
# storing results for comparison
results = []

In [ ]:
# looping through different model configurations to find the best one (large portions of code are autofilled by google colab's gemini)
for hidden_layers in hidden_layers_list:
  for hidden_units in hidden_units_list:
    print(f"Training model with {hidden_layers} hidden layers and {hidden_units} hidden units")
    deep_model = DeepNetwork(input_size = X_train.shape[1] * X_train.shape[2], hidden_layers = hidden_layers, hidden_units = hidden_units, output_size = y_train.shape[1]).to(device)
    train_losses, test_losses = train_deep_model(deep_model, train_loader, test_loader, epochs = 5, lr = 0.001)

    final_test = test_losses[-1]
    results.append((hidden_layers, hidden_units, final_test))

    if final_test < best_loss:
      best_loss = final_test
      best_config = (hidden_layers, hidden_units)

Training model with 1 hidden layers and 32 hidden units
Epoch 1, Train Loss: 2.5650275096125985, Test Loss: 1.6143889210440896
Epoch 2, Train Loss: 1.6046539092886036, Test Loss: 1.5984236543828791
Epoch 3, Train Loss: 1.5437473140913864, Test Loss: 1.4823730046098882
Epoch 4, Train Loss: 1.4411429591562557, Test Loss: 1.456859534437006
Epoch 5, Train Loss: 1.4074589356608775, Test Loss: 1.5018726587295532
Training model with 1 hidden layers and 64 hidden units
Epoch 1, Train Loss: 2.7267442229150354, Test Loss: 1.4953981041908264
Epoch 2, Train Loss: 1.317820024216312, Test Loss: 1.529812373898246
Epoch 3, Train Loss: 1.2036489063295825, Test Loss: 1.3158719593828374
Epoch 4, Train Loss: 1.124148160561748, Test Loss: 1.3027807149020108
Epoch 5, Train Loss: 1.1471550786632232, Test Loss: 1.3226164850321682
Training model with 1 hidden layers and 128 hidden units
Epoch 1, Train Loss: 3.536744223243889, Test Loss: 1.4291713562878696
Epoch 2, Train Loss: 1.3208346065433545, Test Loss: 1.3

In [ ]:
# printing results to find the simplest model
best_loss = best_loss * 1.05

# sort by the layers and units for the simplest model
sort_results = sorted(results, key=lambda x: (x[0], x[1]))

# finding the simplest model that is within the threshold
for hidden_layers, hidden_units, test_loss in sort_results:
  if test_loss <= best_loss:
   simplest_config = (hidden_layers, hidden_units)
   simplest_loss = test_loss
   break

# printing results
print(f"The simplest model has {simplest_config[0]} hidden layer(s) and {simplest_config[1]} hidden units")

The simplest model has 1 hidden layer(s) and 64 hidden units


In [ ]:
# definining the function to evaluate the model
def evaluate_model(model, test_loader):
  model.eval()
  y_true = []
  y_pred = []

  with torch.no_grad():
    for batch_x, batch_y in test_loader:
      batch_x, batch_y = batch_x.to(device), batch_y.to(device)
      outputs = model(batch_x)
      _, predicted = torch.max(outputs, 1)

      y_true.extend(batch_y.argmax(dim=1).cpu().numpy())
      y_pred.extend(predicted.cpu().numpy())

    # commpting metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_true, y_pred)

    # printing results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1, conf_matrix

In [ ]:
# loading the best model
best_model = DeepNetwork(input_size = X_train.shape[1] * X_train.shape[2], hidden_layers = best_config[0], hidden_units = best_config[1], output_size = y_train.shape[1]).to(device)

In [ ]:
# retraining the best model
train_losses, test_losses = train_deep_model(best_model, train_loader, test_loader, epochs = 5, lr = 0.001)

# evaluate the best model
evaluate_model(best_model, test_loader)

Epoch 1, Train Loss: 2.1833126723081215, Test Loss: 1.4851169911297886
Epoch 2, Train Loss: 1.3230576665922142, Test Loss: 1.3261074748906223
Epoch 3, Train Loss: 1.1715473292887897, Test Loss: 1.4478619748895818
Epoch 4, Train Loss: 1.0964956345229313, Test Loss: 1.3739987178282305
Epoch 5, Train Loss: 1.070602262842244, Test Loss: 1.2783523960547014
Accuracy: 0.4939
Precision: 0.5120
Recall: 0.4939
F1 Score: 0.4940


(0.4938716654650324,
 0.5120296223240731,
 0.4938716654650324,
 0.4939515456512623,
 array([[266,  52,  24,  38,  99],
        [ 37,  91,  12,  26,  69],
        [ 26,  13,  40,  29,  50],
        [ 24,  24,  23, 106,  51],
        [ 33,  28,  14,  30, 182]]))

# **C. Compare Results**